### 使用随机森林计算特征重要度
随机森林是常用的衡量特征重要性的方法。
计算正例经过的节点，使用经过节点的样本数目、经过节点的gini系数和等指标。或者随机替换一列数据，重新建立决策树，计算模型的正确率的变换，从而考虑这一列的特征重要度。
特征重要度的判断依据可以是：

    1）经过节点的样本数目
    2）经过节点的gini系数和
    3）随机替换一列特征，重新构树，计算模型正确率变化

### 孤立森林（Isolation Forest）
从根节点要叶子节点的路径长度，可以度量样本是不是噪声的一个指标。
随机选择特征、随机选择分割点，生成一定深度的决策树，若干颗iTree组成iForest
    
    1）计算iTree中样本x从跟到叶子节点的长度f(x)
    2）计算iForest中f(x)的总和F(x)
若样本x是异常值，它应该在大多数的iTree中很快到达叶子，即F(x)较小。

### 提升（boosting）
给定输入向量x和输出向量y组成的若干训练样本(x1, y1)，(x2, y2)...(xn, yn)，目标是找到近似函数F^(x)，使得损失函数L(y, F(x))的损失函数最小。

$L(y, F(x))$的典型定义：
    
$L(y, F(x)) = \frac{1}{2}(y-F(x))^2$ （假设y服从高斯分布）

$L(y, F(x)) = |y-F(x)|$ （假设y服从拉普拉斯分布）

假定最优函数为F* (x)，即：

$F^*(x) = argminE_{(x, y)}[L(y, F(x))]$

假定F(x)是一族基函数$f_{i}(X)$的加权和

$F(x) = sum_{i=1}^M r_{i}f_{i}(x) + const $

### 提升算法框架
梯度提升方法寻找最优解F(x)，使得损失函数在训练集上的期望最小。

首先，给定常函数$F_{0}(x)$:
$F_{0}(x) = argmin_{r} \sum_{i=1}^n L(y_{i}, r)$

以贪心的思路扩展得到$F_{m}(x)$

$F_{m}(x) = F_{m-1}(x) + argmin_{f \in{H}} \sum_{i=1}^n L(y_{i}, F_{m-1}(x) + f(x))$

### Xgboost核心推导过程

### 考虑使用二阶导信息
求第t颗决策树的目标函数，$J(f)$是GBDT或Xgboost的损失函数。

$f_{t}(x_{i})$就是第t棵树对样本$x_{i}$的预测值，预测值本身是残差部分。

![image.png](./imgs/img01.png)
其中目标函数的解释，
![image.png](./imgs/img02.jpg)

最开始一棵树，然后如果是MAE选用中位数预测y，如果是MSE选用平均数预测y，然后第二棵树就在这个基础上再次求导。

### 正则项的定义
决策树的复杂度可考虑叶结点数和叶权值，如使用叶结点总数和叶权值平方和的加权。
这里$w_{j}$为叶子节点的预测值
$\Omega(f_{t}) = \Upsilon T + \frac{1}{2} \lambda \sum_{j=1}^T w_{j}^{2}$

### 目标函数计算
![image.png](./imgs/img03.png)

n--------样本个数

$y_{i}$--------第i个样本的真实值

$y_{i}^{^{t-1}}$--------第t-1个决策树的第i个样本的预测值

T--------当前叶子节点个数

$\sum_{i\in I_{j}}g_{i}$--------位于j号叶子的所有样本的梯度的和

$f_{t}(x_{i})$--------第i个样本的预测值

$w_{q}(x_{i})$--------第i个样本在哪个叶子（与上面的等价）

### 目标函数继续化简
![image.png](./imgs/img04.png)

### 构造树的结构
问：对于当前节点，如何进行子树划分？
![image.png](./imgs/img05.png)
借鉴ID3/C4.5/CART的做法，使用贪心法：
    
    1）对于某可行划分，计算划分后的J(f)
    2)对于所有可行划分，选择J(f)降低到最小的分割点

以上就是XGBoost的核心推导过程。
相对于传统的GBDT，XGBoost使用了二阶信息，可以更快地在训练集上收敛。

### 总结
gboost采用第一棵树取均值，第二颗树真实值和第二颗树的预测值的差值重新构建树，树的损失函数根据Taylor展式进行转换，新的目标函数包含一阶导和二阶导信息。令损失函数相对于$w_{j}$的导数为0可以求出最小化损失函数各个叶子节点的预测值。通过将预测值带入损失函数可求得损失函数的最小值，容易计算节点分裂前后损失函数的差值。Xgboost就是采用最大化这个差值为准则来进行决策树的构建。